In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
from einops import rearrange, repeat
from torch.nn import MultiheadAttention
from models import *
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

E:\PROJECTS\MultimodalToolBox\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
E:\PROJECTS\MultimodalToolBox\.venv\Lib\site-packages\timm\models\layers\__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
E:\PROJECTS\MultimodalToolBox\.venv\Lib\site-packages\timm\models\hub.py:4: FutureWarning: Importing from timm.models.hub is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)


In [2]:
# Mock data
batch_size = 3
seq_len = 5
emb_dim = 64
vocab_size = 301
input_ids = torch.tensor([[100, 200, 300, 300, 0],
             [22, 33, 44, 0, 0],
             [66, 55, 66, 30, 0]], dtype=torch.long).to(device)
pad_mask = input_ids.eq(0)
emb_layer = nn.Embedding(vocab_size, emb_dim).to(device)
inputs = emb_layer(input_ids)

input_img = torch.randn((3, 3, 224, 224)).to(device)
pad_mask = pad_mask.unsqueeze(1).expand(batch_size, 224*224, seq_len)

## Cross-Attention

In [3]:
cross_att = CrossAttention(emb_dim, in_channel=3, att_dropout=0.1).to(device)
outputs, att_weights = cross_att(input_img, inputs, pad_mask)
print(outputs.shape)
print(att_weights.shape)

torch.Size([3, 3, 224, 224])
torch.Size([3, 50176, 5])


## Cross-MultiHead-Attention

In [4]:
cross_multi_att = CrossMultiAttention(emb_dim, in_channel=3, num_heads=8, att_dropout=0.1).to(device)
outputs, att_weights = cross_multi_att(input_img, inputs, pad_mask)
print(outputs.shape)
print(att_weights.shape)

torch.Size([3, 3, 224, 224])
torch.Size([3, 8, 50176, 5])


## VisionTransformer

In [5]:
vit = VisionTransformer(drop_rate=0.1, attn_drop_rate=0.1, drop_path_rate=0.1).to(device)
outputs = vit(input_img)
print(f"outputs: {outputs.shape}")

outputs: torch.Size([3, 197, 768])
